# Prod demo Prediction

In [0]:
import mlflow.spark
from pyspark.sql.functions import col

# 1. Define the model URI and your Volume path
model_name = "workspace.ecommerce.elite_product_classifier"
model_uri = f"models:/{model_name}@champion"
dfs_tmp_path = "/Volumes/workspace/ecommerce/ecommerce_data" # Must be a UC Volume

# 2. LOAD NATIVELY WITH VOLUME PATH
# This provides the temporary storage Spark needs to load the model in UC
loaded_model = mlflow.spark.load_model(model_uri, dfs_tmpdir=dfs_tmp_path)

# 3. Load your Delta Table (The one you just stored)
new_data = spark.table("workspace.ecommerce.mlprepfeatures")

# This ensures the VectorAssembler never sees a null
#clean_data = new_data.na.fill(0, subset=feature_cols)

# 4. RUN PREDICTION
# The Pipeline handles the VectorAssembler automatically
predictions_df = loaded_model.transform(new_data)

# 5. SELECT FINAL COLUMNS
final_report = predictions_df.select(
    "product_id",
    "total_views",
    "v2p_rate",
    "price_mean",
    "is_top_seller",
    col("prediction").alias("is_predicted_top_seller")
)

# 6. Show the winners
display(final_report.filter("is_predicted_top_seller = 1"))

In [0]:
# 6. Show the winners
display(final_report.filter("is_predicted_top_seller = 1"))